<a href="https://colab.research.google.com/github/jeshwanth-A/defi_aiml/blob/main/core.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q langchain langchain-groq numpy pandas scikit-learn torch tensorflow ipywidgets

In [9]:

import os, sys, subprocess


# --- INSTALL DEPENDENCIES FIRST (runs once, silent) ---
def _install_deps():
    """Install LangChain + Groq if not present."""
    try:
        import langchain_groq
        import langchain_core
    except ImportError:
        print("Installing LangChain + Groq...")
        subprocess.check_call(
            [
                sys.executable,
                "-m",
                "pip",
                "install",
                "-q",
                "langchain-core",
                "langchain-groq",
            ]
        )
        print("Done!")


_install_deps()

# --- IMPORTS ---
import json, sqlite3, pickle, warnings

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
warnings.filterwarnings("ignore", category=UserWarning, module="tensorflow")

import numpy as np, pandas as pd
import torch, torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
import tensorflow as tf

tf.get_logger().setLevel("ERROR")
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from datetime import datetime, timedelta
import urllib.request
import ipywidgets as widgets
from IPython.display import display, clear_output
from google.colab import drive

# LangChain + Groq
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage


# ============================================================================
# CONFIG
# ============================================================================
DRIVE_BASE = "/content/drive/MyDrive/defidoza"
WEIGHTS_DIR = f"{DRIVE_BASE}/weights"
CACHE_DB = f"{DRIVE_BASE}/cache.db"
SCALER_PATH = os.path.join(WEIGHTS_DIR, "scaler.pkl")
PYTORCH_WEIGHTS = os.path.join(WEIGHTS_DIR, "pytorch_lstm.pth")
TF_WEIGHTS = os.path.join(WEIGHTS_DIR, "tf_lstm.h5")
RF_WEIGHTS = os.path.join(WEIGHTS_DIR, "rf_model.pkl")
TOKENS = ["uniswap", "bitcoin", "ethereum", "solana", "cardano"]

# --- LLM CONFIG ---
# >>>>>> REPLACE THIS WITH YOUR GROQ API KEY <<<<<<
GROQ_API_KEY = "enter-api-key-here"
GROQ_MODEL = "llama-3.3-70b-versatile"
GROQ_TEMPERATURE = 0.2


# Mount Google Drive and create directories
drive.mount("/content/drive")
os.makedirs(WEIGHTS_DIR, exist_ok=True)
print(f"Storage: {DRIVE_BASE}")


# ============================================================================
# ADMIN LOGGER
# ============================================================================
_admin_logger = None


def set_admin_logger(logger):
    global _admin_logger
    _admin_logger = logger


def admin_log(category, msg):
    """Log to admin output."""
    formatted = f"[{category}] {msg}"
    if _admin_logger:
        _admin_logger(formatted)


# ============================================================================
# CACHE
# ============================================================================
def init_cache():
    conn = sqlite3.connect(CACHE_DB)
    c = conn.cursor()
    c.execute(
        "CREATE TABLE IF NOT EXISTS price_data (id INTEGER PRIMARY KEY, token TEXT, timestamp TEXT, price REAL, volume REAL, fetched_at TEXT)"
    )
    c.execute(
        "SELECT name FROM sqlite_master WHERE type='table' AND name='predictions'"
    )
    if c.fetchone():
        c.execute("PRAGMA table_info(predictions)")
        cols = [row[1] for row in c.fetchall()]
        if "days" not in cols:
            c.execute("DROP TABLE predictions")
            print("Migrated: Dropped old predictions table (missing 'days' column)")
    c.execute(
        "CREATE TABLE IF NOT EXISTS predictions (id INTEGER PRIMARY KEY, token TEXT, model TEXT, days INTEGER, target_date TEXT, predicted REAL, actual REAL, mae REAL, created_at TEXT)"
    )
    conn.commit()
    conn.close()


init_cache()


def save_price_data(token, df):
    conn = sqlite3.connect(CACHE_DB)
    c = conn.cursor()
    c.execute("DELETE FROM price_data WHERE token = ?", (token,))
    t = datetime.now().isoformat()
    for _, r in df.iterrows():
        c.execute(
            "INSERT INTO price_data (token, timestamp, price, volume, fetched_at) VALUES (?,?,?,?,?)",
            (token, str(r["timestamp"]), r["price"], r["volume"], t),
        )
    conn.commit()
    conn.close()
    admin_log("SAVE", f"Price data: {token} ({len(df)} rows)")


def get_cached_data(token, days):
    conn = sqlite3.connect(CACHE_DB)
    c = conn.cursor()
    c.execute(
        "SELECT timestamp, price, volume, fetched_at FROM price_data WHERE token = ? ORDER BY timestamp DESC LIMIT ?",
        (token, days),
    )
    rows = c.fetchall()
    conn.close()
    if not rows:
        admin_log("CACHE", f"Price data ({token}, {days}d): NOT FOUND")
        return None
    age = (datetime.now() - datetime.fromisoformat(rows[0][3])).total_seconds() / 3600
    if age > 24:
        admin_log("CACHE", f"Price data ({token}, {days}d): EXPIRED (age: {age:.1f}h)")
        return None
    admin_log(
        "CACHE",
        f"Price data ({token}, {days}d): FOUND (age: {age:.1f}h, {len(rows)} rows)",
    )
    df = pd.DataFrame(rows, columns=["timestamp", "price", "volume", "fetched_at"])
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    return df.drop("fetched_at", axis=1).sort_values("timestamp").reset_index(drop=True)


def get_cached_prediction(token, model, days, target_date):
    conn = sqlite3.connect(CACHE_DB)
    c = conn.cursor()
    c.execute(
        "SELECT predicted, actual, mae, created_at FROM predictions WHERE token=? AND model=? AND days=? AND target_date=? ORDER BY created_at DESC LIMIT 1",
        (token, model, days, target_date),
    )
    row = c.fetchone()
    conn.close()
    key = f"({token}, {model}, {days}d, {target_date})"
    if not row:
        admin_log("CACHE", f"Prediction {key}: NOT FOUND")
        return None
    age = (datetime.now() - datetime.fromisoformat(row[3])).total_seconds() / 3600
    if age > 24:
        admin_log("CACHE", f"Prediction {key}: EXPIRED (age: {age:.1f}h)")
        return None
    admin_log("CACHE", f"Prediction {key}: FOUND (age: {age:.1f}h)")
    return {"predicted": row[0], "actual": row[1], "mae": row[2]}


def save_prediction(token, model, days, target_date, predicted, actual, mae):
    conn = sqlite3.connect(CACHE_DB)
    c = conn.cursor()
    c.execute(
        "INSERT INTO predictions (token, model, days, target_date, predicted, actual, mae, created_at) VALUES (?,?,?,?,?,?,?,?)",
        (
            token,
            model,
            days,
            target_date,
            predicted,
            actual,
            mae,
            datetime.now().isoformat(),
        ),
    )
    conn.commit()
    conn.close()
    admin_log(
        "SAVE", f"Prediction: {token}/{model}/{days}d/{target_date} -> ${predicted:.2f}"
    )


# ============================================================================
# DATA
# ============================================================================
def fetch_and_parse(token_id="uniswap", days=30):
    url = f"https://api.coingecko.com/api/v3/coins/{token_id}/market_chart?vs_currency=usd&days={days}&interval=daily"
    admin_log("API", f"Fetching {token_id} ({days}d) from CoinGecko...")
    try:
        with urllib.request.urlopen(url) as r:
            data = json.loads(r.read().decode("utf-8"))
        prices, volumes = data.get("prices", []), data.get("total_volumes", [])
        if not prices:
            admin_log("API", f"Fetch {token_id}: NO DATA")
            return None
        df = pd.DataFrame(prices, columns=["timestamp", "price"])
        df["volume"] = [v[1] for v in volumes]
        df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
        admin_log("API", f"Fetched {token_id}: {len(df)} rows")
        return df
    except Exception as e:
        admin_log("API", f"Fetch {token_id}: FAILED ({str(e)[:50]})")
        return None


def smart_fetch(token, days):
    cached = get_cached_data(token, days)
    if cached is not None and len(cached) >= days:
        return cached
    df = fetch_and_parse(token, days)
    if df is not None:
        save_price_data(token, df)
    return df


def preprocess_data(df, fit_scaler=True):
    df = df.copy()
    np.random.seed(42)
    df["sentiment"] = np.random.uniform(-1, 1, size=len(df))
    prices = df["price"].values
    ws = min(5, len(prices))
    if len(prices) >= ws:
        vol = np.std(np.lib.stride_tricks.sliding_window_view(prices, ws), axis=1)
        df["volatility"] = np.pad(vol, (ws - 1, 0), mode="edge")
    else:
        df["volatility"] = 0

    if fit_scaler:
        scaler = MinMaxScaler()
        df[["price", "volume", "sentiment", "volatility"]] = scaler.fit_transform(
            df[["price", "volume", "sentiment", "volatility"]]
        )
        os.makedirs(WEIGHTS_DIR, exist_ok=True)
        with open(SCALER_PATH, "wb") as f:
            pickle.dump(scaler, f)
    else:
        with open(SCALER_PATH, "rb") as f:
            scaler = pickle.load(f)
        df[["price", "volume", "sentiment", "volatility"]] = scaler.transform(
            df[["price", "volume", "sentiment", "volatility"]]
        )

    df["price_lag1"] = df["price"].shift(1)
    return df.dropna(), scaler


def create_sequences(data, seq_len=10):
    seq_len = min(seq_len, len(data) - 1)
    X, y = [], []
    for i in range(len(data) - seq_len):
        X.append(data.iloc[i : i + seq_len].values)
        y.append(data.iloc[i + seq_len]["price"])
    return np.array(X), np.array(y)


# ============================================================================
# MODELS
# ============================================================================
class PricePredictor(nn.Module):
    def __init__(self, input_size=5, hidden_size=50, num_layers=3):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])


def create_tf_model(shape):
    m = Sequential([LSTM(50, input_shape=shape), Dense(1)])
    m.compile(optimizer="adam", loss="mse")
    return m


def create_rf_model():
    return RandomForestRegressor(n_estimators=50, random_state=42, max_depth=10)


def weights_exist(name):
    return os.path.exists(
        {
            "pytorch": PYTORCH_WEIGHTS,
            "tensorflow": TF_WEIGHTS,
            "randomforest": RF_WEIGHTS,
        }.get(name, "")
    )


def get_trained_models():
    return [m for m in ["pytorch", "tensorflow", "randomforest"] if weights_exist(m)]


def load_model(name, input_size=5):
    admin_log("MODEL", f"Loading {name}...")
    if name == "pytorch":
        m = PricePredictor(input_size=input_size)
        m.load_state_dict(torch.load(PYTORCH_WEIGHTS, map_location="cpu"))
        m.eval()
        admin_log("MODEL", f"Loaded {name} (input_size={input_size})")
        return m
    elif name == "tensorflow":
        m = tf.keras.models.load_model(TF_WEIGHTS, compile=False)
        admin_log("MODEL", f"Loaded {name}")
        return m
    elif name == "randomforest":
        with open(RF_WEIGHTS, "rb") as f:
            m = pickle.load(f)
        admin_log("MODEL", f"Loaded {name}")
        return m
    return None


# ============================================================================
# TRAINING
# ============================================================================
def train_pytorch(X, y, epochs, log):
    log(f"Training PyTorch ({epochs} epochs)...")
    m = PricePredictor(input_size=X.shape[2])
    dev = "cuda" if torch.cuda.is_available() else "cpu"
    m = m.to(dev)
    opt = torch.optim.Adam(m.parameters(), lr=0.001)
    crit = nn.MSELoss()
    Xt = torch.tensor(X, dtype=torch.float32).to(dev)
    yt = torch.tensor(y, dtype=torch.float32).unsqueeze(1).to(dev)
    loss = None
    for e in range(epochs):
        m.train()
        loss = crit(m(Xt), yt)
        opt.zero_grad()
        loss.backward()
        opt.step()
        if (e + 1) % 10 == 0:
            log(f"  Epoch {e + 1}/{epochs} Loss: {loss.item():.6f}")
    os.makedirs(WEIGHTS_DIR, exist_ok=True)
    torch.save(m.state_dict(), PYTORCH_WEIGHTS)
    log(f"  Saved: {PYTORCH_WEIGHTS}")
    return loss.item() if loss else 0.0


def train_tensorflow(X, y, epochs, log):
    log(f"Training TensorFlow ({epochs} epochs)...")
    m = create_tf_model((X.shape[1], X.shape[2]))

    class CB(tf.keras.callbacks.Callback):
        def on_epoch_end(self, e, logs=None):
            if (e + 1) % 10 == 0:
                log(f"  Epoch {e + 1}/{epochs} Loss: {logs['loss']:.6f}")

    m.fit(
        X,
        y,
        epochs=epochs,
        batch_size=max(1, len(X) // 10),
        verbose=0,
        callbacks=[CB()],
    )
    os.makedirs(WEIGHTS_DIR, exist_ok=True)
    m.save(TF_WEIGHTS)
    log(f"  Saved: {TF_WEIGHTS}")
    return m.evaluate(X, y, verbose=0)


def train_randomforest(X, y, log):
    log("Training RandomForest...")
    Xf = X.reshape(X.shape[0], -1)
    m = create_rf_model()
    m.fit(Xf, y)
    os.makedirs(WEIGHTS_DIR, exist_ok=True)
    with open(RF_WEIGHTS, "wb") as f:
        pickle.dump(m, f)
    score = m.score(Xf, y)
    log(f"  Saved: {RF_WEIGHTS} (R2: {score:.4f})")
    return 1 - score


# ============================================================================
# INFERENCE
# ============================================================================
def inverse_transform_price(scaled_price, scaler):
    """Convert normalized price back to USD."""
    dummy = np.array([[scaled_price, 0, 0, 0]])
    return scaler.inverse_transform(dummy)[0, 0]


def run_inference(name, X, scaler):
    m = load_model(name, input_size=X.shape[2] if len(X.shape) > 2 else X.shape[1])
    if m is None:
        return None
    if name == "pytorch":
        dev = "cuda" if torch.cuda.is_available() else "cpu"
        m = m.to(dev)
        with torch.no_grad():
            pred_scaled = (
                m(torch.tensor(X, dtype=torch.float32).to(dev))
                .cpu()
                .numpy()
                .flatten()[-1]
            )
    elif name == "tensorflow":
        pred_scaled = m.predict(X, verbose=0).flatten()[-1]
    elif name == "randomforest":
        pred_scaled = m.predict(X.reshape(X.shape[0], -1))[-1]
    else:
        return None

    pred_usd = inverse_transform_price(pred_scaled, scaler)
    admin_log(
        "MODEL",
        f"Inference {name}: {pred_scaled:.4f} (scaled) -> ${pred_usd:.2f} (USD)",
    )
    return pred_usd


def get_actual_price(token, target_date):
    target_dt = datetime.strptime(target_date, "%Y-%m-%d")
    if target_dt.date() > datetime.now().date():
        return None
    conn = sqlite3.connect(CACHE_DB)
    c = conn.cursor()
    c.execute(
        "SELECT price FROM price_data WHERE token = ? AND DATE(timestamp) = DATE(?) LIMIT 1",
        (token, target_date),
    )
    row = c.fetchone()
    conn.close()
    if row:
        return row[0]
    days_ago = (datetime.now() - target_dt).days + 5
    df = fetch_and_parse(token, min(days_ago, 365))
    if df is not None:
        save_price_data(token, df)
        df["date"] = df["timestamp"].dt.date
        match = df[df["date"] == target_dt.date()]
        if len(match) > 0:
            return match.iloc[0]["price"]
    return None


# ============================================================================
# LLM TOOLS (LANGCHAIN CORE - NO AGENT EXECUTOR NEEDED)
# ============================================================================
def _normalize_token(token):
    if not token:
        return None
    t = str(token).strip().lower()
    aliases = {
        "btc": "bitcoin",
        "eth": "ethereum",
        "sol": "solana",
        "ada": "cardano",
        "uni": "uniswap",
    }
    t = aliases.get(t, t)
    return t if t in TOKENS else None


def _normalize_days(days, default=30):
    try:
        d = int(days)
    except Exception:
        d = default
    return max(1, d)


def _normalize_date(value):
    if not value:
        return datetime.now().date().isoformat()
    if isinstance(value, datetime):
        return value.date().isoformat()
    if hasattr(value, "strftime"):
        try:
            return value.strftime("%Y-%m-%d")
        except Exception:
            pass
    v = str(value).strip().lower()
    if v in ["today", "now"]:
        return datetime.now().date().isoformat()
    if v == "tomorrow":
        return (datetime.now().date() + timedelta(days=1)).isoformat()
    for fmt in ("%Y-%m-%d", "%Y/%m/%d", "%m/%d/%Y", "%d/%m/%Y"):
        try:
            return datetime.strptime(v, fmt).date().isoformat()
        except Exception:
            continue
    return datetime.now().date().isoformat()


def _normalize_model(model):
    if not model:
        return "all"
    m = str(model).strip().lower()
    return m if m in ["pytorch", "tensorflow", "randomforest", "all"] else "all"


def _predict_with_models(token, days, target_date, model):
    if model != "all" and not weights_exist(model):
        return {
            "error": "Model weights not found",
            "model": model,
            "trained_models": get_trained_models(),
        }

    models = get_trained_models() if model == "all" else [model]
    if not models:
        return {
            "error": "No trained models. Use Train button first.",
            "trained_models": [],
        }

    results = []
    for m in models:
        cached = get_cached_prediction(token, m, days, target_date)
        if cached:
            results.append({"model": m, "cached": True, **cached})
            continue

        df = smart_fetch(token, days + 10)
        if df is None or len(df) < 10:
            results.append({"model": m, "error": "Not enough data"})
            continue

        try:
            df_proc, scaler = preprocess_data(df, fit_scaler=False)
        except Exception:
            results.append({"model": m, "error": "Scaler not found. Train first."})
            continue

        features = df_proc[["price", "volume", "sentiment", "volatility", "price_lag1"]]
        X, _ = create_sequences(features, min(10, len(features) // 2))
        if len(X) == 0:
            results.append({"model": m, "error": "No sequences"})
            continue

        pred = run_inference(m, X, scaler)
        if pred is None:
            results.append({"model": m, "error": "Inference failed"})
            continue

        actual = get_actual_price(token, target_date)
        mae = abs(pred - actual) if actual is not None else None
        save_prediction(token, m, days, target_date, float(pred), actual, mae)
        results.append(
            {
                "model": m,
                "predicted_usd": round(float(pred), 2),
                "actual_usd": round(float(actual), 2) if actual is not None else None,
                "mae": round(float(mae), 2) if mae is not None else None,
            }
        )

    return {
        "token": token,
        "days": days,
        "target_date": target_date,
        "model": model,
        "results": results,
    }


# Define tools with @tool decorator
@tool
def list_supported_tokens() -> str:
    """Return the list of supported cryptocurrency token IDs."""
    return json.dumps({"tokens": TOKENS})


@tool
def check_model_status() -> str:
    """Check which ML models are trained and available for predictions."""
    return json.dumps(
        {
            "trained_models": get_trained_models(),
            "weights": {
                "pytorch": weights_exist("pytorch"),
                "tensorflow": weights_exist("tensorflow"),
                "randomforest": weights_exist("randomforest"),
            },
        }
    )


@tool
def get_current_price(token: str) -> str:
    """Get the current/latest USD price for a cryptocurrency token.

    Args:
        token: Token name like 'bitcoin', 'ethereum', 'solana', 'cardano', 'uniswap' (or btc, eth, sol, ada, uni)
    """
    t = _normalize_token(token)
    if not t:
        return json.dumps(
            {"error": f"Unsupported token '{token}'", "supported_tokens": TOKENS}
        )
    df = smart_fetch(t, 2)
    if df is None or len(df) == 0:
        return json.dumps({"error": "Could not fetch price data"})
    row = df.iloc[-1]
    return json.dumps(
        {
            "token": t,
            "price_usd": round(float(row["price"]), 2),
            "timestamp": str(row["timestamp"]),
        }
    )


@tool
def get_price_history(token: str, days: int = 30) -> str:
    """Get recent price history for a cryptocurrency token.

    Args:
        token: Token name like 'bitcoin', 'ethereum', 'solana', 'cardano', 'uniswap'
        days: Number of days of history (default 30)
    """
    t = _normalize_token(token)
    if not t:
        return json.dumps(
            {"error": f"Unsupported token '{token}'", "supported_tokens": TOKENS}
        )
    d = _normalize_days(days)
    df = smart_fetch(t, d)
    if df is None or len(df) == 0:
        return json.dumps({"error": "Could not fetch price data"})
    tail = df.tail(5)
    recent = [
        {
            "date": str(r["timestamp"].date()),
            "price_usd": round(float(r["price"]), 2),
        }
        for _, r in tail.iterrows()
    ]
    return json.dumps(
        {
            "token": t,
            "days_requested": d,
            "data_points": int(len(df)),
            "latest_price_usd": round(float(df.iloc[-1]["price"]), 2),
            "recent_prices": recent,
        }
    )


@tool
def predict_price(
    token: str, days: int = 30, target_date: str = "", model: str = "all"
) -> str:
    """Predict future price for a cryptocurrency token using trained ML models.

    Args:
        token: Token name like 'bitcoin', 'ethereum', 'solana', 'cardano', 'uniswap'
        days: Days of historical data to use (default 30)
        target_date: Date to predict for (YYYY-MM-DD format, or 'today', 'tomorrow')
        model: Which model to use - 'pytorch', 'tensorflow', 'randomforest', or 'all'
    """
    t = _normalize_token(token)
    if not t:
        return json.dumps(
            {"error": f"Unsupported token '{token}'", "supported_tokens": TOKENS}
        )
    d = _normalize_days(days)
    td = _normalize_date(target_date)
    m = _normalize_model(model)
    result = _predict_with_models(t, d, td, m)
    return json.dumps(result)


# All available tools
ALL_TOOLS = [
    list_supported_tokens,
    check_model_status,
    get_current_price,
    get_price_history,
    predict_price,
]

# Tool lookup by name
TOOL_MAP = {t.name: t for t in ALL_TOOLS}


class DefiDozaAgent:
    """Simple tool-calling agent using Groq directly (no AgentExecutor needed)."""

    SYSTEM_PROMPT = """You are DefiDoza, a helpful crypto forecasting assistant.

You have access to these tools:
- list_supported_tokens: Get the list of tokens you can work with
- check_model_status: See which ML models are trained
- get_current_price: Get current USD price for a token
- get_price_history: Get recent price history
- predict_price: Run ML prediction for a token's future price

Guidelines:
- When users ask about prices, use get_current_price
- When users ask for predictions/forecasts, use predict_price
- When users ask about trends or history, use get_price_history
- When users ask what tokens are supported, use list_supported_tokens
- When users ask about trained models, use check_model_status
- If prediction fails because no models are trained, tell the user to use the Train button first
- Be concise and helpful in your responses
- Format prices nicely (e.g., "$42,123.45")"""

    def __init__(self, api_key: str):
        self.llm = ChatGroq(
            model=GROQ_MODEL,
            api_key=api_key,
            temperature=GROQ_TEMPERATURE,
        )
        self.llm_with_tools = self.llm.bind_tools(ALL_TOOLS)
        self.messages = [{"role": "system", "content": self.SYSTEM_PROMPT}]

    def _execute_tool(self, tool_call) -> str:
        """Execute a tool call and return the result."""
        tool_name = tool_call["name"]
        tool_args = tool_call.get("args", {})

        if tool_name not in TOOL_MAP:
            return json.dumps({"error": f"Unknown tool: {tool_name}"})

        try:
            result = TOOL_MAP[tool_name].invoke(tool_args)
            return result
        except Exception as e:
            return json.dumps({"error": str(e)})

    def ask(self, question: str) -> str:
        """Process a user question, potentially using tools."""
        # Add user message
        self.messages.append({"role": "user", "content": question})

        # Get initial response
        response = self.llm_with_tools.invoke(self.messages)

        # Check if there are tool calls
        if hasattr(response, "tool_calls") and response.tool_calls:
            # Add assistant message with tool calls
            self.messages.append(response)

            # Execute each tool call
            for tool_call in response.tool_calls:
                tool_result = self._execute_tool(tool_call)
                # Add tool result as a tool message
                self.messages.append(
                    ToolMessage(content=tool_result, tool_call_id=tool_call["id"])
                )

            # Get final response after tool execution
            final_response = self.llm_with_tools.invoke(self.messages)
            self.messages.append(final_response)
            return final_response.content
        else:
            # No tool calls, just return the response
            self.messages.append(response)
            return response.content

    def clear_history(self):
        """Clear conversation history (keep system prompt)."""
        self.messages = [{"role": "system", "content": self.SYSTEM_PROMPT}]


def build_ask_agent():
    """Build the DefiDoza agent."""
    api_key = os.getenv("GROQ_API_KEY", GROQ_API_KEY)
    if not api_key or api_key == "your-groq-api-key-here":
        return (
            None,
            "Set GROQ_API_KEY! Replace the placeholder in the script or set as environment variable.",
        )

    try:
        agent = DefiDozaAgent(api_key)
        return agent, None
    except Exception as e:
        return None, f"Failed to create agent: {str(e)}"


# ============================================================================
# UI
# ============================================================================
class UI:
    def __init__(self):
        init_cache()
        self.ask_agent = None
        self.build()
        set_admin_logger(self.admin_log_msg)

    def build(self):
        self.out = widgets.Output(
            layout={"height": "350px", "overflow": "auto", "border": "1px solid #999"}
        )
        self.admin_out = widgets.Output(
            layout={"height": "350px", "overflow": "auto", "border": "1px solid #999"}
        )

        # Main buttons
        self.btn_predict = widgets.Button(
            description="Predict", layout={"width": "100px"}
        )
        self.btn_ask = widgets.Button(
            description="Ask",
            layout={"width": "100px"},
        )
        self.btn_train = widgets.Button(description="Train", layout={"width": "100px"})
        self.btn_clear = widgets.Button(description="Clear", layout={"width": "80px"})
        self.btn_toggle_log = widgets.ToggleButton(
            value=True, description="Show", layout={"width": "60px"}
        )
        self.btn_clear_admin = widgets.Button(
            description="Clear", layout={"width": "80px"}
        )
        self.btn_toggle_admin = widgets.ToggleButton(
            value=True, description="Show", layout={"width": "60px"}
        )

        # Train widgets
        self.t_token = widgets.Dropdown(
            options=TOKENS,
            value="uniswap",
            description="Token:",
            layout={"width": "200px"},
        )
        self.t_days = widgets.IntText(
            value=30, description="Days:", layout={"width": "150px"}
        )
        self.t_epochs = widgets.IntText(
            value=50, description="Epochs:", layout={"width": "150px"}
        )
        self.t_model = widgets.RadioButtons(
            options=[
                ("PyTorch", "pytorch"),
                ("TensorFlow", "tensorflow"),
                ("RandomForest", "randomforest"),
                ("All", "all"),
            ],
            value="pytorch",
            description="Model:",
        )
        self.btn_run_train = widgets.Button(
            description="Start", layout={"width": "100px"}
        )
        self.btn_back_train = widgets.Button(
            description="Back", layout={"width": "100px"}
        )

        # Predict widgets
        self.p_token = widgets.Dropdown(
            options=TOKENS,
            value="uniswap",
            description="Token:",
            layout={"width": "200px"},
        )
        self.p_days = widgets.IntText(
            value=30, description="Days:", layout={"width": "150px"}
        )
        self.p_date = widgets.DatePicker(
            description="Date:", value=datetime.now().date(), layout={"width": "200px"}
        )
        self.p_model = widgets.RadioButtons(
            options=self.model_opts(), value=self.default_model(), description="Model:"
        )
        self.btn_run_pred = widgets.Button(description="Run", layout={"width": "100px"})
        self.btn_back_pred = widgets.Button(
            description="Back", layout={"width": "100px"}
        )

        # Ask widgets
        self.ask_input = widgets.Textarea(
            value="",
            placeholder="Ask about prices, predictions, models, or tokens...\nExamples:\n- What's Bitcoin trading at?\n- Predict Ethereum for tomorrow\n- Which models are trained?",
            description="",
            layout={"width": "450px", "height": "100px"},
        )
        self.btn_run_ask = widgets.Button(
            description="Send", button_style="primary", layout={"width": "100px"}
        )
        self.btn_back_ask = widgets.Button(
            description="Back", layout={"width": "100px"}
        )
        self.btn_clear_chat = widgets.Button(
            description="New Chat", layout={"width": "100px"}
        )

        # Status
        self.status = widgets.Label(value=self.status_text())

        # Panels
        self.menu = widgets.VBox(
            [
                self.status,
                widgets.HBox([self.btn_predict, self.btn_ask, self.btn_train]),
            ]
        )
        self.train_panel = widgets.VBox(
            [
                widgets.Label("TRAINING"),
                self.t_token,
                self.t_days,
                self.t_epochs,
                self.t_model,
                widgets.HBox([self.btn_run_train, self.btn_back_train]),
            ],
            layout={"display": "none"},
        )
        self.pred_panel = widgets.VBox(
            [
                widgets.Label("PREDICTION"),
                self.p_token,
                self.p_days,
                self.p_date,
                self.p_model,
                widgets.HBox([self.btn_run_pred, self.btn_back_pred]),
            ],
            layout={"display": "none"},
        )
        self.ask_panel = widgets.VBox(
            [
                widgets.Label("ASK DefiDoza (powered by Groq)"),
                self.ask_input,
                widgets.HBox(
                    [self.btn_run_ask, self.btn_clear_chat, self.btn_back_ask]
                ),
            ],
            layout={"display": "none"},
        )
        self.main = widgets.VBox(
            [
                self.menu,
                self.train_panel,
                self.pred_panel,
                self.ask_panel,
                widgets.HBox(
                    [widgets.Label("LOG:"), self.btn_toggle_log, self.btn_clear]
                ),
                self.out,
                widgets.HBox(
                    [
                        widgets.Label("ADMIN LOG:"),
                        self.btn_toggle_admin,
                        self.btn_clear_admin,
                    ]
                ),
                self.admin_out,
            ]
        )

        # Events
        self.btn_predict.on_click(lambda b: self.show("pred"))
        self.btn_ask.on_click(lambda b: self.show("ask"))
        self.btn_train.on_click(lambda b: self.show("train"))
        self.btn_clear.on_click(lambda b: self.out.clear_output())
        self.btn_clear_admin.on_click(lambda b: self.admin_out.clear_output())
        self.btn_toggle_log.observe(self.toggle_log, names="value")
        self.btn_toggle_admin.observe(self.toggle_admin, names="value")
        self.btn_back_train.on_click(lambda b: self.show("menu"))
        self.btn_back_pred.on_click(lambda b: self.show("menu"))
        self.btn_back_ask.on_click(lambda b: self.show("menu"))
        self.btn_run_train.on_click(self.do_train)
        self.btn_run_pred.on_click(self.do_predict)
        self.btn_run_ask.on_click(self.do_ask)
        self.btn_clear_chat.on_click(self.do_clear_chat)

    def model_opts(self):
        opts = []
        for n, k in [
            ("PyTorch", "pytorch"),
            ("TensorFlow", "tensorflow"),
            ("RandomForest", "randomforest"),
        ]:
            opts.append((f"{n} [{'OK' if weights_exist(k) else 'X'}]", k))
        if get_trained_models():
            opts.append(("All", "all"))
        return opts

    def default_model(self):
        t = get_trained_models()
        return t[0] if t else "pytorch"

    def status_text(self):
        t = get_trained_models()
        models = f"Trained: {', '.join(t)}" if t else "No models trained"
        return models

    def show(self, panel):
        self.menu.layout.display = "block" if panel == "menu" else "none"
        self.train_panel.layout.display = "block" if panel == "train" else "none"
        self.pred_panel.layout.display = "block" if panel == "pred" else "none"
        self.ask_panel.layout.display = "block" if panel == "ask" else "none"
        if panel == "menu":
            self.status.value = self.status_text()
            self.btn_predict.disabled = len(get_trained_models()) == 0
        if panel == "pred":
            self.p_model.options = self.model_opts()

    def toggle_log(self, change):
        if change["new"]:
            self.out.layout.display = "block"
            self.btn_toggle_log.description = "Show"
        else:
            self.out.layout.display = "none"
            self.btn_toggle_log.description = "Hide"

    def toggle_admin(self, change):
        if change["new"]:
            self.admin_out.layout.display = "block"
            self.btn_toggle_admin.description = "Show"
        else:
            self.admin_out.layout.display = "none"
            self.btn_toggle_admin.description = "Hide"

    def log(self, msg):
        with self.out:
            print(msg)

    def admin_log_msg(self, msg):
        with self.admin_out:
            print(msg)

    def do_train(self, b):
        with self.out:
            clear_output()
            token, days, epochs, model = (
                self.t_token.value,
                self.t_days.value,
                self.t_epochs.value,
                self.t_model.value,
            )
            self.log(f"Config: {token}, {days} days, {epochs} epochs, {model}")

            df = smart_fetch(token, days)
            if df is None or len(df) < 10:
                self.log("Error: Not enough data")
                return
            self.log(f"Data: {len(df)} rows")

            df_proc, _ = preprocess_data(df, fit_scaler=True)
            features = df_proc[
                ["price", "volume", "sentiment", "volatility", "price_lag1"]
            ]
            X, y = create_sequences(features, min(10, len(features) // 2))
            if len(X) == 0:
                self.log("Error: Not enough sequences")
                return

            split = max(1, int(0.8 * len(X)))
            X_train, y_train = X[:split], y[:split]
            self.log(f"Sequences: {len(X_train)} train")

            if model in ["pytorch", "all"]:
                train_pytorch(X_train, y_train, epochs, self.log)
            if model in ["tensorflow", "all"]:
                train_tensorflow(X_train, y_train, epochs, self.log)
            if model in ["randomforest", "all"]:
                train_randomforest(X_train, y_train, self.log)

            self.log("Done.")
            self.status.value = self.status_text()
            self.btn_predict.disabled = len(get_trained_models()) == 0

    def do_predict(self, b):
        with self.out:
            clear_output()
            token, days, target_date, model = (
                self.p_token.value,
                self.p_days.value,
                self.p_date.value.strftime("%Y-%m-%d"),
                self.p_model.value,
            )
            self.log(f"Config: {token}, {days} days, date {target_date}, model {model}")

            models = (
                get_trained_models()
                if model == "all"
                else ([model] if weights_exist(model) else [])
            )
            if not models:
                self.log("Error: No trained model")
                return

            results = []
            for m in models:
                self.log(f"\n{m}:")
                cached = get_cached_prediction(token, m, days, target_date)
                if cached:
                    self.log("  Using cache")
                    results.append({"model": m, **cached})
                    continue

                df = smart_fetch(token, days + 10)
                if df is None or len(df) < 10:
                    self.log("  Error: Not enough data")
                    continue

                try:
                    df_proc, scaler = preprocess_data(df, fit_scaler=False)
                except:
                    self.log("  Error: No scaler found")
                    continue

                features = df_proc[
                    ["price", "volume", "sentiment", "volatility", "price_lag1"]
                ]
                X, y = create_sequences(features, min(10, len(features) // 2))
                if len(X) == 0:
                    self.log("  Error: No sequences")
                    continue

                pred = run_inference(m, X, scaler)
                actual = get_actual_price(token, target_date)
                mae = abs(pred - actual) if actual else None
                save_prediction(token, m, days, target_date, float(pred), actual, mae)
                results.append(
                    {"model": m, "predicted": pred, "actual": actual, "mae": mae}
                )

            self.log("\n" + "=" * 40)
            self.log(f"{'Model':<12} {'Predicted':<12} {'Actual':<12} {'MAE':<10}")
            self.log("-" * 40)
            for r in results:
                p = f"${r['predicted']:.2f}" if r["predicted"] else "N/A"
                a = f"${r['actual']:.2f}" if r["actual"] else "Future"
                e = f"${r['mae']:.2f}" if r["mae"] else "--"
                self.log(f"{r['model']:<12} {p:<12} {a:<12} {e:<10}")

            maes = [r for r in results if r["mae"]]
            if maes:
                best = min(maes, key=lambda x: x["mae"])
                self.log(f"\nBest: {best['model']} (MAE: ${best['mae']:.2f})")

    def _ensure_ask_agent(self):
        if self.ask_agent is not None:
            return self.ask_agent
        self.log("Initializing Ask agent...")
        agent, err = build_ask_agent()
        if not agent:
            self.log(f"Ask unavailable: {err}")
            return None
        self.ask_agent = agent
        self.log("Ask agent ready!")
        return agent

    def do_ask(self, b):
        with self.out:
            q = self.ask_input.value.strip()
            if not q:
                self.log("Type a question first.")
                return
            agent = self._ensure_ask_agent()
            if not agent:
                return
            self.log(f"\nYou: {q}")
            self.log("Thinking...")
            try:
                response = agent.ask(q)
            except Exception as e:
                self.log(f"Error: {str(e)[:300]}")
                return
            self.log(f"DefiDoza: {response}")
            self.ask_input.value = ""

    def do_clear_chat(self, b):
        if self.ask_agent:
            self.ask_agent.clear_history()
        with self.out:
            clear_output()
            self.log("Chat history cleared. Starting fresh conversation.")

    def display(self):
        display(self.main)


# ============================================================================
# RUN
# ============================================================================
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    print(f"Device: CUDA ({gpu_name})")
else:
    print("Device: CPU")

print("LangChain + Groq: Ready")
ui = UI()
ui.display()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Storage: /content/drive/MyDrive/defidoza
Device: CUDA (Tesla T4)
LangChain + Groq: Ready
